In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

In [2]:
# # import cdr data
# cdr_data = pd.read_csv("data/cdr_all_feats_data_new_split.csv")
# gcn_data = pd.read_csv("data/gcn_all_feats_data_new_split.csv")

In [3]:
# this is the train index
test_index = 0

In [4]:
deepcdr_preds = np.load("..//data//deepcdr_preds_val_and_test_" +str(test_index) + ".npy")

In [5]:
dualgcn_preds = np.load("..//data//dualgcn_complete_preds_val_and_test_" +str(test_index) + ".npy")

In [6]:
deepcdr_preds.shape, dualgcn_preds.shape

((11544, 26), (11544, 26))

In [7]:
np.mean(deepcdr_preds[:,0] == dualgcn_preds[:,0])

1.0

In [8]:

# valid_half = 0

In [9]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [10]:
chunks = np.array_split(list(range(0,deepcdr_preds.shape[0])), 2)

In [11]:
# this is the half for the stacker training
half = 0

In [12]:
train_items = chunks[half]

In [13]:
test_items = list(set(list(range(0,deepcdr_preds.shape[0]))).difference(train_items))

In [14]:
# dualgcn_preds[:,0]

In [15]:
# cdr_data.columns = ['True_response', 'Predicted_y_cdr']

In [16]:
valid_deepcdr = deepcdr_preds[train_items,1:]
test_deepcdr = deepcdr_preds[test_items,1:]

In [17]:
valid_dualgcn = dualgcn_preds[train_items,1:]
test_dualgcn = dualgcn_preds[test_items,1:]

In [18]:
# valid_dualgcn.shape

In [19]:
valid_y = deepcdr_preds[train_items,0]
test_y = deepcdr_preds[test_items,0]

In [20]:
test_y.mean()

2.094244087144837

In [21]:
test_y.std()

2.8136855131635183

In [22]:
# stacker_data_train.shape

In [23]:
reps = valid_deepcdr.shape[1]

In [24]:
# valid_deepcdr

In [25]:
%%time
catch_preds = np.zeros((test_dualgcn.shape[0],reps))
catch_weights = np.zeros((reps,2))
for i in range(0,valid_deepcdr.shape[1]):
    print(i)
    stacker_data_train = np.hstack((valid_deepcdr[:,i].reshape(-1,1), valid_dualgcn[:,i].reshape(-1,1)))
    stacker_data_test = np.hstack((test_deepcdr[:,i].reshape(-1,1), test_dualgcn[:,i].reshape(-1,1)))
    reg = LinearRegression()
    reg.fit(stacker_data_train, valid_y)
    # reg.
    y_preds = reg.predict(stacker_data_test)
    res = pearsonr(test_y.tolist(), y_preds.tolist())
    catch_preds[:,i] = y_preds
    catch_weights[i,0], catch_weights[i,1] = reg.coef_[0]/(reg.coef_[0]+reg.coef_[1]), reg.coef_[1]/(reg.coef_[0]+reg.coef_[1])
    # print(res[0])
    # print(np.sqrt(mean_squared_error(test_y, y_preds)))
    # plt.scatter(y_preds.reshape(-1,1), test_y, alpha = 0.1)
    # plt.axline((0,0), slope = 1, c = "black")
    # plt.show()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
CPU times: user 189 ms, sys: 789 ms, total: 979 ms
Wall time: 82.1 ms


In [26]:
# test_y

In [27]:
catch_preds.shape

(5772, 25)

In [28]:
li, ui = np.percentile(catch_preds, q = [2.5, 97.5], axis = 1)[0,:], np.percentile(catch_preds, q = [2.5, 97.5], axis = 1)[1,:]

In [29]:
test_coverage = np.mean((test_y.reshape(-1,1) >= li.reshape(-1,1)) & (test_y.reshape(-1,1) <= ui.reshape(-1,1)))
test_coverage

0.43347193347193347

In [30]:
test_width = np.mean(ui.reshape(-1,1) - li.reshape(-1,1))
test_width

1.2210323021678768

In [31]:
catch_preds.mean(1).shape

(5772,)

In [32]:
test_y.shape

(5772,)

In [33]:
test_pearsonr = pearsonr(test_y, catch_preds.mean(1))
test_pearsonr[0]

0.9139605354671096

In [34]:
test_rmse = np.sqrt(np.mean((catch_preds.mean(1).reshape(-1,1)-test_y.reshape(-1,1))**2))
test_rmse

1.1437700964050153

In [35]:
catch_weights

array([[0.55514262, 0.44485738],
       [0.57913955, 0.42086045],
       [0.579281  , 0.420719  ],
       [0.56057213, 0.43942787],
       [0.54354903, 0.45645097],
       [0.56637057, 0.43362943],
       [0.56750444, 0.43249556],
       [0.54026892, 0.45973108],
       [0.5327982 , 0.4672018 ],
       [0.55774662, 0.44225338],
       [0.54919396, 0.45080604],
       [0.56503372, 0.43496628],
       [0.53612455, 0.46387545],
       [0.55335106, 0.44664894],
       [0.5498429 , 0.4501571 ],
       [0.55559566, 0.44440434],
       [0.52920572, 0.47079428],
       [0.56573769, 0.43426231],
       [0.55572045, 0.44427955],
       [0.5401512 , 0.4598488 ],
       [0.54824861, 0.45175139],
       [0.55942013, 0.44057987],
       [0.55644367, 0.44355633],
       [0.54328501, 0.45671499],
       [0.54973041, 0.45026959]])

In [36]:
test_cdr_weight, test_gcn_weight = np.mean(catch_weights,0)[0], np.mean(catch_weights,0)[1]
test_cdr_weight, test_gcn_weight

(0.5535783126475694, 0.4464216873524307)

In [37]:
np.std(catch_weights,0)

array([0.01271419, 0.01271419])

In [38]:
# save the metrics for later\
col_name = ['test_coverage', 'test_width', 'test_pearsonr', 'test_rmse', 'test_cdr_weight', 'test_gcn_weight' ]
col_vals = [test_coverage, test_width, test_pearsonr[0], test_rmse, test_cdr_weight, test_gcn_weight]

In [39]:
fold_idx = 2

In [40]:
Stacker_test_results = pd.DataFrame({'parameter': col_name, 'value': col_vals})
save_name = 'test_parameters_all_features_stacker_complete_for_test_fold_' + str(fold_idx) + '.csv'
save_path = '../MEnKF_fold_results/Fold_2/'
Stacker_test_results.to_csv(save_path + save_name, index = False)

In [41]:
Stacker_test_results

,parameter,value
0,test_coverage,0.433472
1,test_width,1.221032
2,test_pearsonr,0.913961
3,test_rmse,1.143770
4,test_cdr_weight,0.553578
5,test_gcn_weight,0.446422
